# Covid-19’s Impact on Airport Traffic

## Import Packages

In [4]:
import pandas as pd
import numpy as np
import datetime
import os
import json

In [6]:
import fiona
import geopandas as gpd
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
import folium

In [7]:
#%pip install bokeh
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar, NumeralTickFormatter
from bokeh.palettes import brewer

from bokeh.io.doc import curdoc
from bokeh.models import Slider, HoverTool, Select
from bokeh.layouts import widgetbox, row, column

## Read in Datasets (Partly Preprocessed with PySpark)

In [22]:
# aggregated flight data - only need the US data
df_flight = pd.read_csv('flight_result.csv',names=['Month','Airport','City','State','Country','Percentage'])
df_flight=df_flight.loc[df_flight['Country']=='United States of America (the)']
df_flight

,Month,Airport,City,State,Country,Percentage
0,3,Boston Logan International,Boston,Massachusetts,United States of America (the),80.6875
2,3,Charlotte Douglas International,Charlotte,North Carolina,United States of America (the),73.0000
3,3,Chicago OHare International,Chicago,Illinois,United States of America (the),77.8125
4,3,Dallas/Fort Worth International,Grapevine,Texas,United States of America (the),70.9375
5,3,Daniel K. Inouye International,Urban Honolulu,Hawaii,United States of America (the),95.0000
...,...,...,...,...,...,...
213,10,Miami International,Miami Springs,Florida,United States of America (the),66.6250
216,10,Newark Liberty International,Newark,New Jersey,United States of America (the),98.9375
217,10,San Francisco International,South San Francisco,California,United States of America (the),51.6250
219,10,Seattle-Tacoma International,SeaTac,Washington,United States of America (the),68.0625


In [10]:
# aggregated covid stats data
df_cases = pd.read_csv('case_final.csv')
df_cases.rename(columns={'state':'State'},inplace=True)
df_cases

,State,1,2,3,4,5,6,7,8,9,10,11,12
0,Alabama,0.0,0.0,1063.0,6124.0,11013.0,20277.0,49789.0,38234.0,28272.0,38841.0,55911.0,20353.0
1,Alaska,0.0,0.0,129.0,231.0,124.0,658.0,2574.0,2457.0,2672.0,7510.0,16221.0,4460.0
2,AmericanSamoa,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Arizona,1.0,0.0,1288.0,6366.0,12281.0,59292.0,94782.0,27825.0,16672.0,27439.0,80871.0,37459.0
4,Arkansas,0.0,0.0,523.0,2758.0,3972.0,13524.0,21734.0,18713.0,22473.0,28493.0,45169.0,13565.0
5,California,3.0,9.0,8221.0,41897.0,61821.0,119467.0,269138.0,211919.0,106640.0,113028.0,298121.0,136409.0
6,Colorado,0.0,0.0,2966.0,12318.0,11094.0,6337.0,14094.0,10615.0,13112.0,36814.0,125555.0,27676.0
7,Connecticut,0.0,0.0,3128.0,24572.0,14501.0,4313.0,3296.0,3069.0,4671.0,13657.0,46088.0,10420.0
8,Delaware,0.0,0.0,319.0,4415.0,4764.0,1976.0,3314.0,2641.0,3184.0,4338.0,10703.0,4258.0
9,DistrictofColumbia,0.0,0.0,495.0,3828.0,4478.0,1526.0,1799.0,1866.0,1334.0,1940.0,4286.0,1584.0


In [23]:
# prepare the mega table used for plotting -> add a column cases in the flight df

month_lst = df_flight['Month'].values
state_lst = df_flight['State'].values
merged_list = [(month_lst[i], state_lst[i]) for i in range(0, len(month_lst))] 

def getCases(state,month):
    return df_cases[df_cases['State']==state][month].values[0]

case_lst = []
for tup in merged_list:
    state = tup[1].replace(' ','')
    month = str(tup[0])
    res = getCases(state,month)
    case_lst.append(res)

df_flight['Cases']=case_lst

In [24]:
# the final mega table with all the info we need
df_flight

,Month,Airport,City,State,Country,Percentage,Cases
0,3,Boston Logan International,Boston,Massachusetts,United States of America (the),80.6875,9922.0
2,3,Charlotte Douglas International,Charlotte,North Carolina,United States of America (the),73.0000,1578.0
3,3,Chicago OHare International,Chicago,Illinois,United States of America (the),77.8125,5992.0
4,3,Dallas/Fort Worth International,Grapevine,Texas,United States of America (the),70.9375,3809.0
5,3,Daniel K. Inouye International,Urban Honolulu,Hawaii,United States of America (the),95.0000,204.0
...,...,...,...,...,...,...,...
213,10,Miami International,Miami Springs,Florida,United States of America (the),66.6250,96031.0
216,10,Newark Liberty International,Newark,New Jersey,United States of America (the),98.9375,32611.0
217,10,San Francisco International,South San Francisco,California,United States of America (the),51.6250,113028.0
219,10,Seattle-Tacoma International,SeaTac,Washington,United States of America (the),68.0625,19979.0


## Build Reactive Plot with Bokeh
* reference: https://jimking100.github.io/2019-09-04-Post-3/
* geojson data for US states boundaries: https://eric.clst.org/tech/usgeojson/

In [13]:
#prepare the geo info
us_states_path = 'gz_2010_us_040_00_5m.json'
cur_json = json.load(open(us_states_path, encoding='ISO-8859-1'))
path,ext = os.path.splitext(us_states_path)
new_path =path+"_new"+ext
with open(new_path,"w", encoding='utf-8') as jsonfile:
    json.dump(cur_json,jsonfile,ensure_ascii=False)
us_states = gpd.read_file(new_path, driver='GeoJSON')

In [14]:
# Set the Coordinate Referance System (crs) for projections
# ESPG code 4326 is also referred to as WGS84 lat-long projection
us_states.crs = {'init': 'epsg:4326'}
us_states=us_states.rename(columns={'NAME':'State'})
us_states.drop([1],axis=0,inplace=True)

In [15]:
# Create a function the returns json_data for the month selected by the user
def json_data(selectedMon):
    mon = selectedMon
    
    # Pull selected year from neighborhood summary data
    df_mon = df_flight[df_flight['Month']==mon]
    
    # Merge the GeoDataframe object (sf) with the neighborhood summary data (neighborhood)
    merged = pd.merge(us_states, df_mon, on='State', how='left')
    
    # Bokeh uses geojson formatting, representing geographical features, with json
    # Convert to json
    merged_json = json.loads(merged.to_json())
    
    # Convert to json preferred string-like object 
    json_data = json.dumps(merged_json)
    return json_data

In [16]:
def update_plot(attr, old, new):
    
    # The input yr is the year selected from the slider
    mon = slider.value
    new_data = json_data(mon)
    
    # The input cr is the criteria selected from the select box
    cr = select.value
    input_field = format_df.loc[format_df['verbage'] == cr, 'field'].iloc[0]
    
    # Update the plot based on the changed inputs
    p = make_plot(input_field)
    
    # Update the layout, clear the old document and display the new document
    layout = column(p, widgetbox(select), widgetbox(slider))
    curdoc().clear()
    curdoc().add_root(layout)
    
    # Update the data
    geosource.geojson = new_data

In [17]:
# Create a plotting function
def make_plot(field_name):    
    # Set the format of the colorbar
    min_range = format_df.loc[format_df['field'] == field_name, 'min_range'].iloc[0]
    max_range = format_df.loc[format_df['field'] == field_name, 'max_range'].iloc[0]
    field_format = format_df.loc[format_df['field'] == field_name, 'format'].iloc[0]
    
    # Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
    color_mapper = LinearColorMapper(palette = palette, low = min_range, high = max_range)
    # Create color bar.
    format_tick = NumeralTickFormatter(format=field_format)
    color_bar = ColorBar(color_mapper=color_mapper, label_standoff=18, formatter=format_tick, border_line_color=None, location = (0, 0))
    
    # Create figure object.
    verbage = format_df.loc[format_df['field'] == field_name, 'verbage'].iloc[0]
    p = figure(title = verbage, plot_height = 650, plot_width = 1350, toolbar_location = None)
    p.xgrid.grid_line_color = None
    p.ygrid.grid_line_color = None
    p.axis.visible = False
    
    # Add patch renderer to figure. 
    p.patches('xs','ys', source = geosource, fill_color = {'field' : field_name, 'transform' : color_mapper}, line_color = 'black', line_width = 0.25, fill_alpha = 1)
    
    # Specify color bar layout.
    p.add_layout(color_bar, 'right')
    
    # Add the hover tool to the graph
    p.add_tools(hover)
    return p

In [18]:
#color bar
format_data = [('Cases',0,200000,'0','Newly Increased COVID-19 Cases'),
              ('Percentage',0,100,'0.0','Percentage of Flight Compared to Baseline Period(Jan|Feb)')]
format_df = pd.DataFrame(format_data, columns = ['field' , 'min_range', 'max_range' , 'format', 'verbage'])

In [19]:
geosource = GeoJSONDataSource(geojson = json_data(3))
input_field = 'Cases'

# Define a sequential multi-hue color palette.
palette = brewer['Blues'][8]

# Reverse color order so that dark blue is highest obesity.
palette = palette[::-1]

hover = HoverTool(tooltips = [ ('Airport','@Airport'),('State','@State'),('City', '@City'),('%flights', '@Percentage{,}'),('#new cases', '@Cases{,}')])
p = make_plot(input_field)
                  
# Make a slider object: slider 
slider = Slider(title = 'Month of 2020',start = 3, end = 10, step = 1, value = 3)
slider.on_change('value', update_plot)

# Make a selection object: select
select = Select(title='Select Criteria:', value='Newly Increased COVID-19 Cases', options=['Newly Increased COVID-19 Cases','Percentage of Flight Compared to Baseline Period(Jan|Feb)'])
select.on_change('value', update_plot)

# Make a column layout of widgetbox(slider) and plot, and add it to the current document
# Display the current document
layout = column(p, widgetbox(select),widgetbox(slider))
curdoc().add_root(layout)

In [ ]:
# running this command can create a static plot without the reactive components
show(p)

In [ ]:
#command for a reactive fully working plot: python -m bokeh serve --show demo-notebook.ipynb (run this in virtual env terminal)